In [1]:
import warnings
warnings.filterwarnings("ignore")
import bo_wrangle as wra
import wrangle as w
import functions as f

import pandas as pd
import numpy as np

#splits
from sklearn.model_selection import train_test_split

#visualization
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

#statistical
from scipy.stats import chi2_contingency

#scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score, classification_report

#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier

# Acquire

In [2]:
merged_data = w.left_join_csv('austin_animal_outcomes.csv', 'austin_animal_intakes.csv', 'merged_data.csv')

In [3]:
df = merged_data

# Prepare

In [4]:
def prep_df(df):
    """
    Preprocesses the input DataFrame to prepare it for analysis.

    Parameters:
        df (DataFrame): The input DataFrame to be preprocessed.

    Returns:
        df (DataFrame): The preprocessed DataFrame containing relevant data after filtering and transformations.
        model_df (DataFrame): A model-ready DataFrame, containing one-hot encoded columns for categorical variables.

    Preprocessing Steps:
    1. Lowercase column names and values in the DataFrame.
    2. Remove duplicate 'animal id' entries and drop rows with duplicate 'id'.
    3. Rename certain columns for clarity and consistency.
    4. Filter 'species' to only return 'dog' or 'cat'.
    5. Drop rows with missing values in 'outcome', 'intake_type', 'sex', and 'name' columns.
    6. Replace 'nan' values in 'name' column with 0 and all other names with 1.
    7. Drop rows with 'wildlife' as the 'intake_type'.
    8. Convert 'dob', 'outcome_datetime', and 'intake_datetime' columns to the datetime data type.
    9. Create a new column 'outcome_age' representing the age at the outcome date.
    10. Create 'rel_month' and 'rel_year' columns from the 'outcome_date'.
    11. Rename values in the 'outcome' column based on a mapping dictionary and rename remaining values to 'other'.
    12. Transform 'intake_condition' and 'color' columns using separate functions.
    13. Update data types for 'name' and 'outcome_age' columns.
    14. Drop unnecessary columns from the DataFrame.
    15. Categorize the 'breed' column into 'mix', 'two breeds', and 'single breed'.
    16. Create dummy variables for categorical columns: 'sex', 'intake_type', 'condition', 'species', 'breed', 'primary_color'.
    17. Create the model_df by concatenating boolean columns with dummy_df.

    Note:
    - The 'transform_intake_condition' and 'transform_color' functions are assumed to be defined elsewhere in the code.
    - The specific transformations for some columns (e.g., 'breed') depend on the data and problem domain.
    - The returned model_df is suitable for modeling purposes with categorical variables one-hot encoded.
    """
# lower cases df
    df.columns = df.columns.str.lower()
    df = df.apply(lambda x: x.astype(str).str.lower())
    # returns all dupes
    duplicates = df[df['animal id'].duplicated()]
    # duplicate ids -- that need to drop
    dupe_list = list(duplicates['animal id'].unique())
    # removed dupes --  shape after dropping dupes (124940, 23)
    df = df[~df['animal id'].isin(dupe_list)]
    # rename columns
    new_columns = {
        'datetime_x': 'outcome_datetime',
        'datetime_y': 'intake_datetime',
        'name_y': 'name',
        'breed_y': 'breed',
        'animal type_y': 'species',
        'outcome type': 'outcome',
        'color_y': 'color',
        'sex upon outcome': 'sex',
        'intake type': 'intake_type',
        'date of birth': 'dob',
        'intake condition': 'condition',
        'animal id': 'id'      
    }
    df = df.rename(columns=new_columns)
    # Filter 'species' to only return cats or dogs
    df = df[df['species'].isin(['dog', 'cat'])]
    ### drop nulls
    # drop nan from outcome
    df = df[df.outcome != "nan"]
    # drop nan from intake type
    df = df[df.intake_type != "nan"]
    # drop nan from sex and 
    df = df[~df['sex'].isin(['nan', 'unknown'])]
    # Replace 'nan' values in 'name' column with 0
    df['name'] = df['name'].replace('nan', 0)
    # Replace all other names with 1
    df.loc[df['name'] != 0, 'name'] = 1
    # outlier drops
    # drop wildlife variable from intake type
    df = df[df.intake_type != "wildlife"]
    # fix datatypes
    df['dob'] = pd.to_datetime(df['dob'])
    # change dtype to datetime
    df['outcome_date'] = pd.to_datetime(df['outcome_datetime']).dt.strftime('%m/%d/%Y').astype("datetime64")
    df['intake_date'] = pd.to_datetime(df['intake_datetime']).dt.strftime('%m/%d/%Y').astype("datetime64")
    # create release age
    df['outcome_age'] = (df.outcome_date - df.dob).dt.days
    # Convert 'outcome_date' column to datetime
    df['outcome_date'] = pd.to_datetime(df['outcome_date'])
    # create month and year 
    df["rel_month"] = df['outcome_date'].dt.strftime('%b')
    df["rel_year"] = df['outcome_date'].dt.year
    # age column
    # Define the conditions for each age category
    conditions = [
        (df['outcome_age'] <= 730),
        (df['outcome_age'] >= 731) & (df['outcome_age'] <= 2920),
        (df['outcome_age'] >= 2921)
    ]
    # Define the corresponding values for each age category
    values = ['puppy', 'adult', 'senior']
    # lower cases df
    df.columns = df.columns.str.lower()
    df = df.apply(lambda x: x.astype(str).str.lower())
    # returns all dupes
    duplicates = df[df['id'].duplicated()]
    # duplicate ids -- that need to drop
    dupe_list = list(duplicates['id'].unique())
    # removed dupes --  shape after dropping dupes (124940, 23)
    df = df[~df['id'].isin(dupe_list)]
    # rename columns
    new_columns = {
        'datetime_x': 'outcome_datetime',
        'datetime_y': 'intake_datetime',
        'name_y': 'name',
        'breed_y': 'breed',
        'animal type_y': 'species',
        'outcome type': 'outcome',
        'color_y': 'color',
        'sex upon outcome': 'sex',
        'intake type': 'intake_type',
        'date of birth': 'dob',
        'intake condition': 'condition',
        'animal id': 'id'      
    }
    df = df.rename(columns=new_columns)
    # Filter 'species' to only return cats or dogs
    df = df[df['species'].isin(['dog', 'cat'])]
    ### drop nulls
    # drop nan from outcome
    df = df[df.outcome != "nan"]
    # drop nan from intake type
    df = df[df.intake_type != "nan"]
    # drop nan from sex and 
    df = df[~df['sex'].isin(['nan', 'unknown'])]
    # Replace 'nan' values in 'name' column with 0
    df['name'] = df['name'].replace('nan', 0)
    # Replace all other names with 1
    df.loc[df['name'] != 0, 'name'] = 1
    # outlier drops
    # drop wildlife variable from intake type
    df = df[df.intake_type != "wildlife"]
    # change dtype to datetime
    df['outcome_date'] = pd.to_datetime(df['outcome_datetime']).dt.strftime('%m/%d/%Y').astype("datetime64")
    df['intake_date'] = pd.to_datetime(df['intake_datetime']).dt.strftime('%m/%d/%Y').astype("datetime64")
    # Convert 'outcome_date' column to datetime
    df['outcome_date'] = pd.to_datetime(df['outcome_date'])
    # create month and year 
    df["rel_month"] = df['outcome_date'].dt.strftime('%b')
    df["rel_year"] = df['outcome_date'].dt.year
    # Create a mapping dictionary for renaming
    mapping = {
        'return to owner': 'adoption',
        'rto-adopt': 'adoption'
    }
    # Rename values in 'outcome' column based on the mapping dictionary
    df['outcome'] = df['outcome'].replace(mapping)
    # Rename remaining values to 'other'
    df.loc[~df['outcome'].isin(['adoption', 'transfer']), 'outcome'] = 'other'
    # create intake columns and colors
    df = w.transform_intake_condition(df)
    df = w.transform_color(df)
    
    # update dtypes
    df.name = df.name.astype('int')
    df.outcome_age = df.outcome_age.astype('int')
    df['dob'] = pd.to_datetime(df['dob'])
    # drop these columns
    df = df.drop(columns=["id","name_x", "monthyear_x", "animal type_x",
                     "sex upon intake", "age upon outcome", "breed_x",
                     "color_x", "monthyear_y", "found location", "age upon intake",
                          "outcome subtype", "intake_datetime", "outcome_datetime", "outcome_date", "intake_date"])
    # Rename values in 'breed' column
    df.loc[df['breed'].str.contains('mix|domestic shorthair|domestic medium hair|domestic longhair', case=False), 'breed'] = 'mix'
    df.loc[df['breed'].str.contains('/', na=False), 'breed'] = 'two breeds'
    df.loc[~df['breed'].isin(['two breeds', 'mix']), 'breed'] = 'single breed'
    dummy_df = pd.get_dummies(df[[ 'sex','intake_type', 'condition',
                             'species', 'breed', 'primary_color']],
                          drop_first=True)
    
    bool_df = df[['outcome','name', 'is_tabby', 'mix_color']]
    model_df = pd.concat([bool_df, dummy_df], axis=1)
    return df, model_df


In [5]:
df, model_df = prep_df(df)

In [6]:
df.head()

,dob,outcome,sex,name,intake_type,condition,species,breed,outcome_age,rel_month,rel_year,primary_color,is_tabby,mix_color
0,2017-05-02,adoption,neutered male,1,owner surrender,normal,cat,mix,736,may,2019,brown,1,1
1,2017-07-12,adoption,neutered male,1,stray,normal,dog,mix,371,jul,2018,white,0,1
5,2014-03-12,transfer,intact male,1,stray,medical attention,cat,mix,6,mar,2014,orange,1,0
8,2018-03-01,adoption,neutered male,1,owner surrender,normal,dog,two breeds,797,may,2020,white,0,1
9,2022-05-04,adoption,neutered male,1,stray,normal,cat,mix,76,jul,2022,brown,1,0


In [7]:
model_df.head()

,outcome,name,is_tabby,mix_color,sex_intact male,sex_neutered male,sex_spayed female,intake_type_euthanasia request,intake_type_owner surrender,intake_type_public assist,...,primary_color_gray,primary_color_lilac point,primary_color_lynx point,primary_color_orange,primary_color_red,primary_color_sable,primary_color_seal point,primary_color_tan,primary_color_tricolor,primary_color_white
0,adoption,1,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,adoption,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,transfer,1,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,adoption,1,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
9,adoption,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Split

In [8]:
def get_xy(model_df):
    '''
    This function generates X and y for train, validate, and test to use : X_train, y_train, X_validate, y_validate, X_test, y_test = get_xy()

    '''
    train, validate, test = w.split_data(model_df,'outcome')

    X_train = train.drop(['outcome'], axis=1)
    y_train = train.outcome
    X_validate = validate.drop(['outcome'], axis=1)
    y_validate = validate.outcome
    X_test = test.drop(['outcome'], axis=1)
    y_test = test.outcome
    return X_train,y_train,X_validate,y_validate,X_test,y_test

In [9]:
# Split the model_df
X_train,y_train,X_validate,y_validate,X_test,y_test = get_xy(model_df)

train -> (66047, 34), 60.0%
validate -> (22016, 34),20.0%
test -> (22016, 34), 20.0%


In [10]:
X_test.head()

,name,is_tabby,mix_color,sex_intact male,sex_neutered male,sex_spayed female,intake_type_euthanasia request,intake_type_owner surrender,intake_type_public assist,intake_type_stray,...,primary_color_gray,primary_color_lilac point,primary_color_lynx point,primary_color_orange,primary_color_red,primary_color_sable,primary_color_seal point,primary_color_tan,primary_color_tricolor,primary_color_white
177983,1,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
172889,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
60818,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
39597,1,0,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
15749,1,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# split data for string df so can concat back on for prediction worksheet
X_train_str,y_train_str,X_validate_str,y_validate_str,X_test_str,y_test_str = get_xy(df)

train -> (66047, 14), 60.0%
validate -> (22016, 14),20.0%
test -> (22016, 14), 20.0%


In [12]:
X_test_str.head()

,dob,sex,name,intake_type,condition,species,breed,outcome_age,rel_month,rel_year,primary_color,is_tabby,mix_color
177983,2018-04-30,neutered male,1,stray,normal,dog,single breed,1107,may,2021,black,0,0
172889,2015-10-17,intact male,1,owner surrender,sick,dog,mix,15,nov,2015,brown,0,0
60818,2020-02-02,spayed female,1,stray,injured,dog,mix,88,apr,2020,black,0,0
39597,2014-05-05,intact male,1,stray,normal,dog,two breeds,8,may,2014,white,0,1
15749,2021-06-01,neutered male,1,stray,normal,cat,mix,198,dec,2021,brown,1,0


# Run Model

In [13]:
baseline_df = f.get_baseline(y_train)

In [14]:
baseline_df

,Baseline,Metric,Score
0,"0 adoption Name: outcome, dtype: object",Accuracy,0.638727


In [15]:
# Create a Gradient Boosting Classifier object
model = GradientBoostingClassifier()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred_test = model.predict(X_test)

# Evaluate the model on the test data
accuracy_test = model.score(X_test, y_test)
print("Test Accuracy:", accuracy_test)

Test Accuracy: 0.7989189680232558


# Create Predictions CSV Merged

In [17]:
# Create a Gradient Boosting Classifier object
model = GradientBoostingClassifier()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred_test = model.predict(X_test)

# Create a dataframe for the predictions
predictions_df = pd.DataFrame({'actual': y_test, 'predicted': y_pred_test})

# Merge the two dataframes
merged_df = pd.concat([X_test, X_test_str, predictions_df], axis=1)

# Save the merged dataframe to a CSV file
merged_df.to_csv('merged_predictions.csv', index=False)

In [24]:
X_test.shape

(22016, 33)